EDITORWARS

[algospot의 EDITORWARS 링크](https://algospot.com/judge/problem/read/EDITORWARS)  
  
### 문제  
에디터 전쟁은 가장 유명한 자유 소프트웨어 텍스트 편집기인 vi와 Emacs 중 어느 쪽이 더 우월한가를 놓고 인터넷에서 자주 벌어지는 논쟁을 말합니다. 이 논쟁에 참여하는 사람들은 서로 자신이 사용하는 편집기의 장점을 찬양하고 (“vi는 동작도 빠르고, 빠른 편집을 가능하게 한다고”, “Emacs는 LISP을 통해 확장 가능하다고”) 다른 편집기를 헐뜯곤 (“vi-vi-vi는 666이잖아! vi는 악마의 편집기야”, “Emacs는 좋은 운영 체제지. 좋은 편집기가 없는 것만 빼면 완벽해”) 합니다.  
  
모든 회원들이 vi 혹은 Emacs를 사용하는 프로그래밍 동호회에서 연말 파티를 개최하려 합니다. 서로 다른 편집기를 사용하는 사람들이 파티에 함께 참가하면 싸움이 나기 때문에 vi를 사용하는 사람들만 오는 파티, Emacs를 사용하는 사람들만 오는 파티를 따로 하기로 했습니다. 이를 위해 지금까지 모든 회원들이 쓴 댓글을 모아 이들을 두 종류로 분류했습니다.  
  
1. 상호 인정: 이 유형의 댓글은 댓글을 쓴 사람과 원글을 쓴 사람이 같은 편집기를 쓴다는 사실을 의미합니다. 예로 “아이고 이런 편집기를 쓰시다니 뭘 아는 분이네” 혹은 “역시 편집기는 xxx가 짱이죠” 등이 있겠지요.  
2. 상호 비방: 이 유형의 댓글은 댓글을 쓴 사람과 원글을 쓴 사람이 다른 편집기를 쓴다는 사실을 의미합니다. 예로 “그런 편집기 쓰면 손가락에 마비가 올 것 같네요” 혹은 “훠어이 악마의 편집기는 물러가라” 등이 있겠지요.  
  
이것만으로는 물론 각 사용자가 어떤 편집기를 쓰는지는 알 수 없지만, 우선 서둘러 장소를 예약해야 하기 때문에 이 정보만으로 파티에 올 수 있는 최대 인원을 알아야 합니다. 댓글 정보가 주어질 때 이 댓글 정보 중 모순되는 것은 없는지 확인하고, 모순되는 것이 없을 때 한 파티의 가능한 최대 인원을 계산하는 프로그램을 작성하세요.  
  
### 출력  
- 댓글에 주어진 정보 중 모순이 없는 경우 한 파티에 올 수 있는 사람의 최대 수를 “MAX PARTY SIZE IS x”의 형태로 출력합니다.  
- 댓글들에 주어진 정보 중 모순이 있는 경우, 모순이 처음으로 발생하는 댓글이 몇 번인지를 “CONTRADICTION AT i” 형태로 출력합니다. 댓글의 번호는 입력에 주어진 순서대로 1부터 시작한다고 합시다.  

solution의  
n : 회원의 수 $(1 \leq n \leq 10000)$  
reply_list : 댓글의 정보를 지니는 리스트 $(1 \leq len(reply_list) \leq 100000)$  
  
각 댓글은 상호 인정, 혹은 상호 비방 댓글입니다. 상호 인정 댓글은 “ACK a b”(0 ≤ a, b < N) 형태로 주어지며 상호 비방 댓글은 “DIS a b” 형태로 주어집니다.  

In [1]:
# Disjoint Set(상호배타적 집합) = Union-find 자료구조

class DisjointSet:
    parent = [] # 해당 인덱스 노드의 부모
    rank = [] # 해당 인덱스 노드의 rank
    enemy = [] # 해당 노드의 적대 관계의 루트 노드
    size = [] # 해당 노드가 루트일 경우 자신 포함 가지는 노드의 개수
    
    def __init__(self, n):
        self.parent = [i for i in range(n)] # 각자 노드의 부모는 자기 자신으로 초기화
        self.rank = [1 for _ in range(n)] # 각자의 rank는 1부터 시작
        self.size = [1 for _ in range(n)] # 각자의 size는 1부터 시작
        self.enemy = [-1 for _ in range(n)] # None일 경우 enemy가 없다는 말이다.
    
    def find(self, u): # u번 노드의 root노트의 인덱스 반환
        if u == self.parent[u]:
            return u
        return self.find(self.parent[u])
    
    def union(self, u, v): # u번 노드와 v번 노드 병합
        if u == -1 or v == -1:
            return max(u,v)
        u, v = self.find(u), self.find(v) # 각자 u,v 노드의 root노드 찾기.
        if u == v: return u
        if(self.rank[u] > self.rank[v]):
            u, v = v, u
        self.parent[u] = v # 랭크가 큰 트리의 root의 subtree로 작은 랭크의 트리가 들어간다.
        self.size[v] += self.size[u] # v노드가 루트가 되면서 v의 size를 더해준다.
        self.size[u] = 0
        if self.rank[u] == self.rank[v]: self.rank[v] += 1
        return v
        # 만약 2개의 트리의 rank가 동일할 때 rank를 1 증가시킨다.
        
    def ack(self, u, v): # u, v는 같은 트리 구성하는 지 여부
        u, v = self.find(u), self.find(v) # u의 루트, v의 루트
        if self.enemy[u] == v: # u의 적이 v와 같다 -> u와 v는 다른 트리 구성
            return False
        a = self.union(u, v) # u, v를 병합
        b = self.union(self.enemy[u], self.enemy[v]) # u의 적, v의 적을 병합
        if b == -1 or a == -1: # 아직 적에 대한 정보가 없을 때
            return True
        self.enemy[a], self.enemy[b] = b, a # a의 적은 b, b의 적은 a
        return True
    
    def dis(self, u, v): # u, v가 다른 트리에 구성되어있는지 여부
        u, v = self.find(u), self.find(v)# u의 루트, v의 루트
        if u == v: return False # u와 v가 동일 -> u와 v는 동일한 트리 구성
        a = self.union(u, self.enemy[v]) # u, v의 적을 병합
        b = self.union(self.enemy[u], v) # v, u의 적을 병합
        if b == -1 or a == -1: # 아직 적에 대한 정보가 없을 때
            return True
        self.enemy[a], self.enemy[b] = b, a # a의 적은 b, b의 적은 a
        return True
    
    def get_maxsize(self): # 최대 사이즈가 될 수 있는 수 반환
        n = len(self.size)
        answer = 0
        enemy = -1 # 적
        node_size = 0 # 현재 노드의 사이즈
        enemy_size = 0 # 현재 노드의 적의 사이즈
        for i in range(n):
            enemy = self.enemy[i]
            if enemy > i: continue
            node_size = self.size[i]
            enemy_size = 0 if enemy == -1 else self.size[enemy]
            answer += max(node_size, enemy_size)
            # 현재 노드 포함된 트리의 사이즈, 적 트리의 사이즈 중 최대값
        return answer

In [2]:
def solution(n, reply_list):
    d_set = DisjointSet(n)
    num = 0
    flag = None
    for intend, a, b in reply_list:
        num += 1
        if intend == "ACK":
            flag = d_set.ack(a, b)
        else: # intend == "DIS"
            flag = d_set.dis(a, b)
        if not flag:
            return "CONTRADICTION AT {}".format(num)
    answer = d_set.get_maxsize()
    return "MAX PARTY SIZE IS {}".format(answer)

In [3]:
n = 4
reply_list = [["ACK", 0, 1],
              ["ACK", 1, 2],
              ["DIS", 1, 3],
              ["ACK", 2, 0]]

In [4]:
solution(n, reply_list)

'MAX PARTY SIZE IS 3'

In [5]:
n = 100
reply_list = [["ACK", 0, 1],
              ["ACK", 1, 2],
              ["ACK", 2, 3],
              ["ACK", 3, 4]]

In [6]:
solution(n, reply_list)

'MAX PARTY SIZE IS 100'

In [7]:
n = 3
reply_list = [["ACK", 0, 1],
              ["ACK", 1, 2],
              ["DIS", 2, 0]]

In [8]:
solution(n, reply_list)

'CONTRADICTION AT 3'

In [9]:
n = 8
reply_list = [["DIS", 0, 1],
              ["ACK", 1, 2],
              ["ACK", 1, 4],
              ["DIS", 4, 3],
              ["DIS", 5, 6],
              ["ACK", 5, 7]]

In [10]:
solution(n, reply_list)

'MAX PARTY SIZE IS 5'